In [0]:
#Importacao das bibliotecas necessarias durante o codigo
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib
from sklearn.datasets import load_breast_cancer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score


In [2]:
!pip install xgboost

In [0]:
#importacao do dataset

bc = load_breast_cancer()
data = pd.DataFrame(bc.data)
data.columns = bc.feature_names
data['TARGET'] = bc.target

In [5]:
data

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,TARGET
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [0]:
#separando as features do target
X =  data.iloc[:,:-1] #features
Y = data.iloc[:,-1] #target

In [8]:
Y

0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: TARGET, Length: 569, dtype: int64

In [0]:
#divisao da base em treino e teste 
semente = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=semente)

In [30]:
#aplicacao do xgboost (sem parametros de otimizacao) no modelo.
model = XGBClassifier(min_child_weight=1.2)
#treinamento
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1.2, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
#predicao da base de teste usando o modelo treinado
y_pred = model.predict(X_test)

In [32]:
#comparacao da previsao na base de teste com o real - metrica utilizada para avaliar o modelo é a acuracia
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 98.40%


In [0]:
#xgb com gridsearch e cross-validation

In [0]:
#parametros para serem testados no gridsearch
max_depth = [3,4,5,6] #profundidade maxima da arvore
min_child_weight = [1.0,2.0] #Soma minima dos pesos das instancias necessarias em um filho
nround = [60,90,100] #numero de arvores
colsample_bytree = [0.7,0.8] #amostras de colunas
subsample = [0.6,0.7,0.8,0.9,1] #amostra de linhas


In [0]:
#lista de parametros tunados
param_grid = dict(max_depth=max_depth, min_child_weight = min_child_weight,nround = nround, colsample_bytree = colsample_bytree, subsample = subsample )
#definicao de particoes para o CrossValidation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7) 

In [0]:
grid_search = GridSearchCV(model,param_grid, n_jobs=-1, cv=kfold, verbose=2)

In [24]:
grid_result = grid_search.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)),

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 240 candidates, totalling 2400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 690 tasks      | elapsed:   34.7s
[Parallel(n_jobs=-1)]: Done 1256 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1986 tasks      | elapsed:  1.7min


Best: 0.965992 using {'colsample_bytree': 0.7, 'max_depth': 6, 'min_child_weight': 1.0, 'nround': 60, 'subsample': 0.8}
0.947571 (0.023318) with: {'colsample_bytree': 0.7, 'max_depth': 3, 'min_child_weight': 1.0, 'nround': 60, 'subsample': 0.6}
0.963360 (0.023771) with: {'colsample_bytree': 0.7, 'max_depth': 3, 'min_child_weight': 1.0, 'nround': 60, 'subsample': 0.7}
0.958097 (0.023815) with: {'colsample_bytree': 0.7, 'max_depth': 3, 'min_child_weight': 1.0, 'nround': 60, 'subsample': 0.8}
0.958097 (0.033484) with: {'colsample_bytree': 0.7, 'max_depth': 3, 'min_child_weight': 1.0, 'nround': 60, 'subsample': 0.9}
0.955466 (0.026191) with: {'colsample_bytree': 0.7, 'max_depth': 3, 'min_child_weight': 1.0, 'nround': 60, 'subsample': 1}
0.947571 (0.023318) with: {'colsample_bytree': 0.7, 'max_depth': 3, 'min_child_weight': 1.0, 'nround': 90, 'subsample': 0.6}
0.963360 (0.023771) with: {'colsample_bytree': 0.7, 'max_depth': 3, 'min_child_weight': 1.0, 'nround': 90, 'subsample': 0.7}
0.95809

[Parallel(n_jobs=-1)]: Done 2400 out of 2400 | elapsed:  2.0min finished


In [0]:
y_pred = grid_result.predict(X_test)

In [35]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 98.40%


In [0]:
auc = roc_auc_score(y_test, y_pred)

In [34]:
print(accuracy)

0.9840425531914894


In [37]:
print(auc)

0.9745762711864407
